# Prediction of Air Quality Index (primary pollutant PM 2.5) of Los Angeles, California

### Import Libraries

In [24]:
import os
import time
import requests # will download the data in form of html
import sys
import csv

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from datetime import datetime

import missingno as msno
import warnings
warnings.filterwarnings('ignore')

### Dataset 1: Los Angeles Intl Airport climate data: https://en.tutiempo.net/climate/ws-722950.html
### Dataset 2: Los Angeles USC Campus Downtown climate data: https://en.tutiempo.net/climate/ws-722874.html

These datasets contain the historical climate data of two weather stations in Los Angeles, CA, United States. The website provides a wide range of data starting from 1973-2019. We want to download the climate data for 12 months through 2010-2020.
    
Interpretation annual average climate values:

- T: Average Temperature (°C)
- TM: Maximum temperature (°C)
- Tm: Minimum temperature (°C)
- SLP: Atmospheric pressure at sea level (hPa)
- H: Average relative humidity (%)
- PP: Total rainfall and / or snowmelt (mm)
- VV: Average visibility (Km)
- V: Average wind speed (Km/h)
- VM: Maximum sustained wind speed (Km/h)
- VG: Maximum speed of wind (Km/h)
- RA: Indicate if there was rain or drizzle (In the monthly average, total days it rained)
- SN: Snow indicator (In the monthly average, total days that snowed)
- TS: Indicates whether there storm (In the monthly average, Total days with thunderstorm)
- FG: Indicates whether there was fog (In the monthly average, Total days with fog)

## Data Collection

### A function that downloads the html data tables in the local machine

In [27]:
def retrieve_html(ws_value, station):
    
    for year in range(2010, 2021): # we want the data for the years from 2010 to 2020
        for month in range (1, 13):
            if (month < 10):
                url = "https://en.tutiempo.net/climate/0{}-{}/ws-{}.html".format(month, year, ws_value) # month is less than 10 means month takes 1 digit 
            else:
                url = "https://en.tutiempo.net/climate/{}-{}/ws-{}.html".format(month, year, ws_value) # here months are 10, 11 and 12
                
            texts  = requests.get(url)
            text_utf = texts.text.encode('utf = 8') # please follow the link if you want to know more about encoding "https://kunststube.net/encoding/"
        
            if not os.path.exists("Data_LA/html_data_LA/{}/{}".format(station, year)):
                os.makedirs("Data_LA/html_data_LA/{}/{}".format(station, year)) # creating the directory here if already not exists 
            with open("Data_LA/html_data_LA/{}/{}/{}.html".format(station, year, month), "wb") as output:
                output.write(text_utf) # writing from text_utf to folders created in Data/html_data/years as html
            
        sys.stdout.flush() # While using stdout, data is stored in buffer memory (for some time or until the memory gets filled) before it gets written to terminal. 
        # Using flush() forces to empty the buffer and write to terminal even before buffer has empty space.        

### Load Dataset 1

In [28]:
# we want to see how long does it take to retrieve the html data files
start_time = time.time()
retrieve_html(722950, 'LAX')
stop_time = time.time()
print("Time taken to retrieve data {}".format(stop_time - start_time))

Time taken to retrieve data 139.79031777381897


### Load Dataset 2

In [29]:
# we want to see how long does it take to retrieve the html data files
start_time = time.time()
retrieve_html(722874, 'USC_Downtown')
stop_time = time.time()
print("Time taken to retrieve data {}".format(stop_time - start_time))

Time taken to retrieve data 137.08098697662354


## Data Wrangling

### Data Wrangling of Dataset 1: Los Angeles Intl Airport climate data

- We will convert all the columns into numeric columns. We will take care of the 'Date' column and convert its type to datetime. 

- We will convert all the '-' into numpy.NaN.

- We want to keep only ['Date', 'T', 'TM', 'Tm', 'SLP', 'H', 'PP', 'VV', 'V', 'VM'] columns as other columns don't provide much data.

- As a very little portion of data (less than 5%) is missing, we will impute the median of the feature/column to replace the numeric missing data

- We will generate the Pandas Profile Report to be able to explore each dataset (month-wise) more.


In [30]:
def monthly_weather_data(month, year, station):

    html_data = pd.read_html('Data_LA/html_data_LA/{}/{}/{}.html'.format(station, year, month))
    df = pd.DataFrame(html_data[2])
    df = df.iloc[:-2]
    df = df.apply(pd.to_numeric, errors='ignore')
    
    if (month in (1, 3, 5, 7, 8, 10, 12)):
        df['Date'] = pd.date_range(start ='{}/01/{}'.format(month, year), end ='{}/31/{}'.format(month, year), freq='D')
    elif (month in (4, 6, 9, 11)):
        df['Date'] = pd.date_range(start ='{}/01/{}'.format(month, year), end ='{}/30/{}'.format(month, year), freq='D')
    elif month == 2 and year in (2012, 2016, 2020):
        df['Date'] = pd.date_range(start ='{}/01/{}'.format(month, year), end ='{}/29/{}'.format(month, year), freq='D')
    elif month == 2 and year in (2010, 2011, 2013, 2014, 2015, 2017, 2018, 2019):
        df['Date'] = pd.date_range(start ='{}/01/{}'.format(month, year), end ='{}/28/{}'.format(month, year), freq='D')

    cols_to_keep = ['Date', 'T', 'TM', 'Tm', 'SLP', 'H', 'PP', 'VV', 'V', 'VM']
    df = df[cols_to_keep]

    df = df.dropna(axis = 'rows', how = 'all')
    
    df = df.replace('-', np.nan)

    col = ['T', 'TM', 'Tm', 'SLP', 'H', 'PP', 'VV', 'V', 'VM']
    df = df.apply(pd.to_numeric, errors='ignore')
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].apply(lambda x: x.strftime('%d-%m-%Y'))
    df = df.dropna(how = 'all')
    
    return df  

In [31]:
def yearly_weather_data(year, station):
    
    a = pd.concat([monthly_weather_data(1, year, station), monthly_weather_data(2, year, station), monthly_weather_data(3, year, station), monthly_weather_data(4, year, station), \
                monthly_weather_data(5, year, station), monthly_weather_data(6, year, station), monthly_weather_data(7, year, station), monthly_weather_data(8, year, station), \
                monthly_weather_data(9, year, station), monthly_weather_data(10, year, station), monthly_weather_data(11, year, station), monthly_weather_data(12, year, station)])
    
    return a

In [32]:
start_time = time.time()

climate_dat_2010_LAX = yearly_weather_data(2010, 'LAX')
climate_dat_2011_LAX = yearly_weather_data(2011, 'LAX')
climate_dat_2012_LAX = yearly_weather_data(2012, 'LAX')
climate_dat_2013_LAX = yearly_weather_data(2013, 'LAX')
climate_dat_2014_LAX = yearly_weather_data(2014, 'LAX')
climate_dat_2015_LAX = yearly_weather_data(2015, 'LAX')
climate_dat_2016_LAX = yearly_weather_data(2016, 'LAX')
climate_dat_2017_LAX = yearly_weather_data(2017, 'LAX')
climate_dat_2018_LAX = yearly_weather_data(2018, 'LAX')
climate_dat_2019_LAX = yearly_weather_data(2019, 'LAX')

stop_time = time.time()
print("Time taken to retrieve data {}".format(stop_time - start_time))

Time taken to retrieve data 3.850970506668091


In [33]:
climate_dat_2019_LAX.head(10)

,Date,T,TM,Tm,SLP,H,PP,VV,V,VM
0,01-01-2019,12.1,17.8,5.0,1016.4,21.0,0.00,16.1,9.1,29.4
1,02-01-2019,11.2,16.7,3.9,1021.3,19.0,0.00,16.1,8.5,18.3
2,03-01-2019,12.4,18.9,3.9,1022.5,33.0,0.00,16.1,8.0,20.6
3,04-01-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05-01-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,06-01-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,07-01-2019,12.5,15.0,9.4,1022.4,82.0,2.03,10.5,15.4,25.9
7,08-01-2019,14.8,21.7,10.6,1020.3,70.0,3.30,16.1,8.0,22.2
8,09-01-2019,13.9,21.7,8.9,1019.8,78.0,0.00,15.1,7.6,31.7
9,10-01-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
start_time = time.time()

climate_dat_2010_Downtown = yearly_weather_data(2010, 'USC_Downtown')
climate_dat_2011_Downtown = yearly_weather_data(2011, 'USC_Downtown')
climate_dat_2012_Downtown = yearly_weather_data(2012, 'USC_Downtown')
climate_dat_2013_Downtown = yearly_weather_data(2013, 'USC_Downtown')
climate_dat_2014_Downtown = yearly_weather_data(2014, 'USC_Downtown')
climate_dat_2015_Downtown = yearly_weather_data(2015, 'USC_Downtown')
climate_dat_2016_Downtown = yearly_weather_data(2016, 'USC_Downtown')
climate_dat_2017_Downtown = yearly_weather_data(2017, 'USC_Downtown')
climate_dat_2018_Downtown = yearly_weather_data(2018, 'USC_Downtown')
climate_dat_2019_Downtown = yearly_weather_data(2019, 'USC_Downtown')

stop_time = time.time()
print("Time taken to retrieve data {}".format(stop_time - start_time))

Time taken to retrieve data 3.8611385822296143


In [35]:
climate_dat_2019_Downtown.head(10)

,Date,T,TM,Tm,SLP,H,PP,VV,V,VM
0,01-01-2019,10.4,17.2,3.3,1016.4,26.0,0.00,15.8,2.0,9.4
1,02-01-2019,9.3,17.2,3.3,1021.3,32.0,0.00,16.1,2.2,11.1
2,03-01-2019,11.3,19.4,3.3,1022.3,35.0,0.00,16.1,1.7,9.4
3,04-01-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05-01-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,06-01-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,07-01-2019,12.5,16.1,7.8,1022.5,81.0,0.76,11.6,5.4,16.5
7,08-01-2019,14.7,22.2,10.0,1020.2,66.0,4.32,14.6,2.8,9.4
8,09-01-2019,13.9,19.4,8.3,1019.7,71.0,0.00,15.8,1.1,9.4
9,10-01-2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*Now we have climate data of different regions of Los Angeles i.e., LAX Airpot and Downtown. We need to combine these two datasets and take a mean according to each date.*

- We will concatenate these DataFrames.
- We will use 'groupby' to find the mean of the climate features according to each date.

In [36]:
def climate_data(df1, df2):
    
    frames = [df1, df2]
    df = pd.concat(frames)
    df = df.groupby(['Date'])['T', 'TM', 'Tm', 'SLP', 'H', 'PP', 'VV', 'V', 'VM'].mean()
    df = df.dropna(axis = 'rows', how = 'all')
    
    return df

In [37]:
climate_data_2010 = climate_data(climate_dat_2010_LAX, climate_dat_2010_Downtown)
climate_data_2011 = climate_data(climate_dat_2011_LAX, climate_dat_2011_Downtown)
climate_data_2012 = climate_data(climate_dat_2012_LAX, climate_dat_2012_Downtown)
climate_data_2013 = climate_data(climate_dat_2013_LAX, climate_dat_2013_Downtown)
climate_data_2014 = climate_data(climate_dat_2014_LAX, climate_dat_2014_Downtown)
climate_data_2015 = climate_data(climate_dat_2015_LAX, climate_dat_2015_Downtown)
climate_data_2016 = climate_data(climate_dat_2016_LAX, climate_dat_2016_Downtown)
climate_data_2017 = climate_data(climate_dat_2017_LAX, climate_dat_2017_Downtown)
climate_data_2018 = climate_data(climate_dat_2018_LAX, climate_dat_2018_Downtown)
climate_data_2019 = climate_data(climate_dat_2019_LAX, climate_dat_2019_Downtown)

In [38]:
climate_data_2010.head(10)

,T,TM,Tm,SLP,H,PP,VV,V,VM
Date,,,,,,,,,
01-01-2010,14.75,20.85,9.20,1023.95,45.0,0.0,16.10,4.20,16.75
01-02-2010,13.70,19.00,10.00,1013.95,70.0,0.0,12.65,4.50,12.95
01-03-2010,14.80,20.15,9.90,1017.85,69.0,0.0,16.10,4.90,18.60
01-04-2010,13.05,16.40,9.50,1013.30,54.5,0.0,16.10,12.50,24.15
01-05-2010,17.15,21.40,12.20,1012.20,48.5,0.0,16.10,11.75,33.25
01-06-2010,17.80,23.35,16.15,1015.30,68.0,0.0,15.45,6.00,14.90
01-07-2010,18.70,22.50,16.30,1010.30,76.5,0.0,7.80,5.85,17.60
01-08-2010,19.30,23.65,16.95,1013.60,70.5,0.0,11.25,6.10,15.80
01-09-2010,19.05,24.70,15.00,1011.70,69.0,0.0,11.75,7.85,17.60


### Dataset 3: https://www.epa.gov/outdoor-air-quality-data/download-daily-data

This dataset provides the daily mean concentartion of PM 2.5 which is the main air pollutant of Los Angeles county. We have downloaded the data from the above-mentioned website for the ten years starting from 2010. 

We want to predict PM 2.5 from the given climate features in dataset 1 of Los Angeles, California . We will use the PM 2.5 data from this dataset.

### Data Wrangling of Dataset 3: The Air Quality Index Data of Los Angeles

- This dataset contains the PM 2.5 data of different regions of Los Angeles. We will take a mean using the 'groupby' method. 
- We will atke care of the 'Date' column



In [39]:
def clean_data_PM(path, year):     
    
    df = pd.read_csv(path) # load dataset

    df['Date'] = pd.to_datetime(df['Date'],  errors='coerce') # changing the type of date column to datetime
    df['Date'] = df['Date'].apply(lambda x: x.strftime('%d-%m-%Y'))

    df = df[['Date', 'Daily Mean PM2.5 Concentration']] # we just need the PM 2.5 column 
    df.rename(columns= {'Daily Mean PM2.5 Concentration':'PM2.5'}, inplace = True)
    df = df.groupby(['Date'])['PM2.5'].mean()

    return df

In [40]:
start_time = time.time()

PM_2010 = clean_data_PM('Data_LA/AQI_LA/aqi2010.csv', 2010)
PM_2011 = clean_data_PM('Data_LA/AQI_LA/aqi2011.csv', 2011)
PM_2012 = clean_data_PM('Data_LA/AQI_LA/aqi2012.csv', 2012)
PM_2013 = clean_data_PM('Data_LA/AQI_LA/aqi2013.csv', 2013)
PM_2014 = clean_data_PM('Data_LA/AQI_LA/aqi2014.csv', 2014)
PM_2015 = clean_data_PM('Data_LA/AQI_LA/aqi2015.csv', 2015)
PM_2016 = clean_data_PM('Data_LA/AQI_LA/aqi2016.csv', 2016)
PM_2017 = clean_data_PM('Data_LA/AQI_LA/aqi2017.csv', 2017)
PM_2018 = clean_data_PM('Data_LA/AQI_LA/aqi2018.csv', 2018)
PM_2019 = clean_data_PM('Data_LA/AQI_LA/aqi2019.csv', 2019)

stop_time = time.time()
print("Time taken to retrieve data {}".format(stop_time - start_time))

Time taken to retrieve data 0.4839920997619629


In [41]:
PM_2019.head(10)

Date
01-01-2019     8.772727
01-02-2019     7.227273
01-03-2019     9.231250
01-04-2019     9.375000
01-05-2019     9.509091
01-06-2019     9.390909
01-07-2019    12.554545
01-08-2019    10.864706
01-09-2019    12.390909
01-10-2019     8.554545
Name: PM2.5, dtype: float64

In [42]:
def concat_weather_PM(df1, df2):
    dff = pd.concat([df1, 
                df2.to_frame()], 
                axis=1)
    df = dff.dropna(axis = 'rows', how = 'all')
    df = df.apply(pd.to_numeric)
    
    return df

In [43]:
df_2010 = concat_weather_PM(climate_data_2010, PM_2010)
df_2011 = concat_weather_PM(climate_data_2011, PM_2011)
df_2012 = concat_weather_PM(climate_data_2012, PM_2012)
df_2013 = concat_weather_PM(climate_data_2013, PM_2013)
df_2014 = concat_weather_PM(climate_data_2014, PM_2014)
df_2015 = concat_weather_PM(climate_data_2015, PM_2015)
df_2016 = concat_weather_PM(climate_data_2016, PM_2016)
df_2017 = concat_weather_PM(climate_data_2017, PM_2017)
df_2018 = concat_weather_PM(climate_data_2018, PM_2018)
df_2019 = concat_weather_PM(climate_data_2019, PM_2019)

In [44]:
df_2019.head(10)

,T,TM,Tm,SLP,H,PP,VV,V,VM,PM2.5
01-01-2019,11.25,17.50,4.15,1016.40,23.5,0.00,15.95,5.55,19.40,8.772727
01-02-2019,13.15,17.50,9.15,1014.80,76.5,22.86,15.75,5.30,12.95,7.227273
01-03-2019,15.15,18.60,13.05,1018.65,82.5,0.38,12.65,6.85,13.85,9.231250
01-04-2019,21.05,28.30,14.45,1014.00,34.5,0.00,16.10,7.50,17.60,9.375000
01-05-2019,15.95,20.25,12.20,1015.65,64.5,0.38,16.10,8.30,22.35,9.509091
01-06-2019,16.75,20.80,15.00,1011.45,75.5,0.00,15.95,7.20,15.00,9.390909
01-07-2019,21.35,27.80,17.20,1014.10,66.5,0.00,15.80,7.85,19.45,12.554545
01-08-2019,21.00,25.80,18.05,1014.80,74.5,0.00,13.50,8.25,20.25,10.864706
01-09-2019,23.50,29.20,20.00,1011.15,72.0,0.00,14.70,7.05,17.65,12.390909
01-10-2019,19.45,24.15,13.90,1010.25,53.0,0.00,16.10,8.60,22.35,8.554545


In [45]:
### How about null values?

dfs = [df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019]

for df in dfs:
    print (df.head(2))
    print (df.isnull().sum())

                T     TM    Tm      SLP     H   PP     VV    V     VM  \
01-01-2010  14.75  20.85   9.2  1023.95  45.0  0.0  16.10  4.2  16.75   
01-02-2010  13.70  19.00  10.0  1013.95  70.0  0.0  12.65  4.5  12.95   

                PM2.5  
01-01-2010  18.644444  
01-02-2010  26.358824  
T        174
TM       174
Tm       174
SLP      174
H        174
PP       174
VV       174
V        174
VM       174
PM2.5      0
dtype: int64
                T    TM    Tm      SLP     H   PP    VV     V     VM  \
01-01-2011  10.55  15.3  5.45  1021.80  38.0  0.0  16.0  5.90  12.05   
01-02-2011  14.05  17.5  8.85  1018.05  66.5  0.0  16.0  4.25  10.95   

                PM2.5  
01-01-2011  22.360000  
01-02-2011  14.511111  
T        178
TM       178
Tm       178
SLP      178
H        178
PP       178
VV       178
V        178
VM       178
PM2.5      0
dtype: int64
                T    TM    Tm      SLP     H   PP     VV    V    VM      PM2.5
01-01-2012  14.30  27.5  7.25  1019.45  59.0  0.0   8.

**Finally, we will concatenate all the DataFrames and drop the index or the 'Date' column.**

In [46]:
frames = [df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019]
df = pd.concat(frames).reset_index(drop = True)
    
print (df.shape)
print (df.isnull().sum())
df.head(10)

(3652, 10)
T        1776
TM       1776
Tm       1776
SLP      1776
H        1776
PP       1776
VV       1776
V        1776
VM       1776
PM2.5       0
dtype: int64


,T,TM,Tm,SLP,H,PP,VV,V,VM,PM2.5
0,14.75,20.85,9.20,1023.95,45.0,0.0,16.10,4.20,16.75,18.644444
1,13.70,19.00,10.00,1013.95,70.0,0.0,12.65,4.50,12.95,26.358824
2,14.80,20.15,9.90,1017.85,69.0,0.0,16.10,4.90,18.60,10.700000
3,13.05,16.40,9.50,1013.30,54.5,0.0,16.10,12.50,24.15,4.788889
4,17.15,21.40,12.20,1012.20,48.5,0.0,16.10,11.75,33.25,8.733333
5,17.80,23.35,16.15,1015.30,68.0,0.0,15.45,6.00,14.90,13.260000
6,18.70,22.50,16.30,1010.30,76.5,0.0,7.80,5.85,17.60,18.730000
7,19.30,23.65,16.95,1013.60,70.5,0.0,11.25,6.10,15.80,17.466667
8,19.05,24.70,15.00,1011.70,69.0,0.0,11.75,7.85,17.60,16.777778
9,23.35,29.10,20.45,1011.50,66.0,0.0,15.65,5.85,15.80,16.320000


### let's take care of the null rows!

In [47]:
df = df.dropna(axis = 0)
df.isnull().sum()

T        0
TM       0
Tm       0
SLP      0
H        0
PP       0
VV       0
V        0
VM       0
PM2.5    0
dtype: int64

In [48]:
df.to_csv('preprocessed_data_LA.csv')